In [15]:
import pandas as pd
import numpy as np
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64

# True Parameters
init_sigmasq, init_range_lon, init_range_lat = 13.059, 0.195, 0.154 
init_advec_lat, init_range_time, init_advec_lon, init_nugget = 0.0218, 1.0, -0.1689, 0.247

init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

true_params_vec = torch.tensor([
    np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), np.log(init_phi4),
    init_advec_lat, init_advec_lon, np.log(init_nugget)
], device=DEVICE, dtype=DTYPE)

In [16]:
true_params_vec

tensor([ 4.2042,  1.6348,  0.4721, -3.2695,  0.0218, -0.1689, -1.3984],
       dtype=torch.float64)

compare heads 0 vs 100 vs 300

In [20]:
df0 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_0heads_1217_v050_LBFGS_18126.0.csv")
df100 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_100heads_1217_v050_LBFGS_18126.0.csv")
df300 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_300heads_1217_v050_LBFGS_18126.0.csv")
df400 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_400heads_1221_v050_LBFGS_18126.0.csv")
df500 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_500heads_1221_v050_LBFGS_18126.0.csv")

df0 = df0.iloc[:,5:(5+7)]
df100 = df100.iloc[:,5:(5+7)]
df300 = df300.iloc[:,5:(5+7)]
df400 = df400.iloc[:,5:(5+7)]
df500 = df500.iloc[:,5:(5+7)]

# mape

import numpy as np

rmse_list = []

for i in range(7):
    # Get the true value for the current parameter
    true_val = true_params_vec[i].cpu().numpy()
    
    # Calculate RMSE for each scenario
    # Formula: sqrt( mean( (estimate - truth)^2 ) )
    rmse0 = np.sqrt(np.mean((df0.iloc[:,i] - true_val)**2))
    rmse100 = np.sqrt(np.mean((df100.iloc[:,i] - true_val)**2))
    rmse300 = np.sqrt(np.mean((df300.iloc[:,i] - true_val)**2))
    rmse400 = np.sqrt(np.mean((df400.iloc[:,i] - true_val)**2))
    rmse500 = np.sqrt(np.mean((df500.iloc[:,i] - true_val)**2))
    
    # Optional: Store for later use
    rmse_list.append([rmse0, rmse100, rmse300, rmse400, rmse500])
    
    print(f'Parameter {i+1}: RMSE 0 heads: {rmse0:.4f}, RMSE 100 heads: {rmse100:.4f}, RMSE 300 heads: {rmse300:.4f}, RMSE 400 heads: {rmse400:.4f}, RMSE 500 heads: {rmse500:.4f}')


Parameter 1: RMSE 0 heads: 0.0546, RMSE 100 heads: 0.0468, RMSE 300 heads: 0.0322, RMSE 400 heads: 0.0227, RMSE 500 heads: 0.0197
Parameter 2: RMSE 0 heads: 0.1233, RMSE 100 heads: 0.0972, RMSE 300 heads: 0.0547, RMSE 400 heads: 0.0381, RMSE 500 heads: 0.0355
Parameter 3: RMSE 0 heads: 0.0161, RMSE 100 heads: 0.0156, RMSE 300 heads: 0.0145, RMSE 400 heads: 0.0127, RMSE 500 heads: 0.0126
Parameter 4: RMSE 0 heads: 0.3337, RMSE 100 heads: 0.3468, RMSE 300 heads: 0.3482, RMSE 400 heads: 0.3338, RMSE 500 heads: 0.3149
Parameter 5: RMSE 0 heads: 0.0055, RMSE 100 heads: 0.0055, RMSE 300 heads: 0.0055, RMSE 400 heads: 0.0057, RMSE 500 heads: 0.0057
Parameter 6: RMSE 0 heads: 0.0091, RMSE 100 heads: 0.0089, RMSE 300 heads: 0.0079, RMSE 400 heads: 0.0087, RMSE 500 heads: 0.0071
Parameter 7: RMSE 0 heads: 0.9404, RMSE 100 heads: 0.7565, RMSE 300 heads: 0.4868, RMSE 400 heads: 0.3126, RMSE 500 heads: 0.2729


In [21]:
import pandas as pd

# 데이터프레임 리스트와 컬럼명 정의
dfs = [df0, df100, df300, df400, df500]
column_names = ['0 heads', '100 heads', '300 heads', '400 heads', '500 heads']
index_names = [f'Parameter {i+1}' for i in range(7)]

# 각 파라미터별(행) 시나리오별(열) STD 계산
std_data = []

for i in range(7):
    # 각 df의 i번째 컬럼에 대해 std 계산
    row_stds = [df.iloc[:, i].std() for df in dfs]
    std_data.append(row_stds)

# 결과를 데이터프레임으로 변환
df_std = pd.DataFrame(std_data, columns=column_names, index=index_names)

print("=== Standard Deviation (STD) Table ===")
print(df_std)

# (선택 사항) 보기 좋게 소수점 포맷팅을 원하시면 아래 주석을 해제하세요
# print(df_std.round(4))

=== Standard Deviation (STD) Table ===
              0 heads  100 heads  300 heads  400 heads  500 heads
Parameter 1  0.015102   0.015192   0.014060   0.012522   0.012664
Parameter 2  0.039441   0.041196   0.039605   0.037098   0.037646
Parameter 3  0.011914   0.011941   0.011910   0.013059   0.013153
Parameter 4  0.068204   0.067811   0.065894   0.036307   0.039860
Parameter 5  0.005549   0.005531   0.005516   0.006056   0.006078
Parameter 6  0.005588   0.005578   0.005144   0.003687   0.003510
Parameter 7  0.363741   0.295666   0.210278   0.132337   0.129633


compare vecc vs dw

In [25]:
dw_re = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_dW_v050_121225_18126.0.csv")
vecc_re = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_1212_v050_LBFGS_18126.0.csv")

#dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_dW_v050_121225_18126.0.csv")
#vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_vecc_1212_v050_LBFGS_18126.0.csv")

# 12 17 25:  vecchia used 100 heads and LOC_ERR_STD = 0.018 

#dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_dW_v050_121725_18126.0.csv")
#vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_vecc_1217_v050_LBFGS_18126.0.csv")

# 12 20 25: vecchia used 400 heads and LOC_ERR_STD = 0.02
#dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_dW_v050_1220_18126.0.csv")
#vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_vecc_1220_v050_LBFGS_18126.0.csv")

# 12 21 25: vecchia used 300 heads and LOC_ERR_STD = 0.02
dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_dW_v050_1221_18126.0.csv")
vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_vecc_1221_v050_LBFGS_18126.0.csv")


dw_re = dw_re.iloc[:,5:(5+7)]
dw_irre = dw_irre.iloc[:,5:(5+7)]
vecc_re = vecc_re.iloc[:,5:(5+7)]
vecc_irre = vecc_irre.iloc[:,5:(5+7)]

true_params_vec

tensor([ 4.2042,  1.6348,  0.4721, -3.2695,  0.0218, -0.1689, -1.3984],
       dtype=torch.float64)

In [27]:
import numpy as np
import pandas as pd

results = []

for i in range(7):
    true_val = true_params_vec[i].cpu().numpy()
    
    # --- REGULAR GRID (RE) ---
    # Bias
    bias_dw_re = np.mean(dw_re.iloc[:,i] - true_val)
    bias_vecc_re = np.mean(vecc_re.iloc[:,i] - true_val)
    # RMSE
    rmse_dw_re = np.sqrt(np.mean((dw_re.iloc[:,i] - true_val)**2))
    rmse_vecc_re = np.sqrt(np.mean((vecc_re.iloc[:,i] - true_val)**2))
    
    # --- IRREGULAR GRID (IRRE) ---
    # Bias
    bias_dw_irre = np.mean(dw_irre.iloc[:,i] - true_val)
    bias_vecc_irre = np.mean(vecc_irre.iloc[:,i] - true_val)
    # RMSE
    rmse_dw_irre = np.sqrt(np.mean((dw_irre.iloc[:,i] - true_val)**2))
    rmse_vecc_irre = np.sqrt(np.mean((vecc_irre.iloc[:,i] - true_val)**2))

    results.append({
        "Param": f"P{i+1}",
        # Regular Results
        "DW_Re Bias": bias_dw_re,
        "Vecc_Re Bias": bias_vecc_re,
        "DW_Re RMSE": rmse_dw_re,
        "Vecc_Re RMSE": rmse_vecc_re,
        # Irregular Results
        "DW_Irre Bias": bias_dw_irre,
        "Vecc_Irre Bias": bias_vecc_irre,
        "DW_Irre RMSE": rmse_dw_irre,
        "Vecc_Irre RMSE": rmse_vecc_irre,
    })

# Create a clean DataFrame
df_full_results = pd.DataFrame(results)

# Display format options (optional, makes it easier to read)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(df_full_results.round(4))

  Param  DW_Re Bias  Vecc_Re Bias  DW_Re RMSE  Vecc_Re RMSE  DW_Irre Bias  Vecc_Irre Bias  DW_Irre RMSE  Vecc_Irre RMSE
0    P1      0.0032        0.0448      0.0218        0.0473       -0.1141         -0.0663        0.1161          0.0691
1    P2      0.0035        0.0845      0.0455        0.0916       -0.0497          0.0201        0.0789          0.0639
2    P3     -0.0007        0.0107      0.0192        0.0158       -0.3141         -0.3158        0.3150          0.3161
3    P4     -0.0105       -0.3468      0.0746        0.3518       -0.0403         -0.3332        0.0704          0.9055
4    P5     -0.0002       -0.0004      0.0078        0.0055       -0.0002          0.0336        0.0100          0.1813
5    P6      0.0011       -0.0072      0.0097        0.0095        0.0014          0.0312        0.0114          0.2425
6    P7     -0.1022       -0.7460      0.2583        0.8411        1.6319          1.5369        1.6323          1.5374


In [28]:
print(f'dw_re_std: \n{dw_re.std()}, {np.mean(dw_re.std())}')
print(f'vecc_re_std: \n{vecc_re.std()}, {np.mean(vecc_re.std())}')
print(f'dw_irre_std: \n{dw_irre.std()}, {np.mean(dw_irre.std())}')
print(f'vecc_irre_std: \n{vecc_irre.std()}, {np.mean(vecc_irre.std())}')

dw_re_std: 
sigma        0.021668
range_lat    0.045596
range_lon    0.019254
advec_lat    0.074195
advec_lon    0.007824
beta         0.009693
nugget       0.238422
dtype: float64, 0.05952193928417541
vecc_re_std: 
sigma        0.015237
range_lat    0.035509
range_lon    0.011638
advec_lat    0.059647
advec_lon    0.005502
beta         0.006311
nugget       0.390368
dtype: float64, 0.07488718028016945
dw_irre_std: 
sigma        0.021844
range_lat    0.062163
range_lon    0.024647
advec_lat    0.058399
advec_lon    0.010172
beta         0.011478
nugget       0.035594
dtype: float64, 0.03204253463059344
vecc_irre_std: 
sigma        0.019700
range_lat    0.061416
range_lon    0.012478
advec_lat    0.852978
advec_lon    0.180459
beta         0.243667
nugget       0.039633
dtype: float64, 0.20147589520987105
